In [1]:
import pandas as pd

In [56]:
train_data = pd.read_csv("C:/Users/aishw/OneDrive/Desktop/Research/Adversarial_Learning/adversarial-learning-for-stance/src/data/twitter_data_naacl/twitter_testA_seenval/train.csv")
test_data = pd.read_csv("C:/Users/aishw/OneDrive/Desktop/Research/Adversarial_Learning/adversarial-learning-for-stance/src/data/twitter_data_naacl/twitter_testA_seenval/test.csv")

In [57]:
train_data.shape

(5418, 8)

In [121]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5418 entries, 0 to 5417
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet            3519 non-null   object 
 1   topic            5418 non-null   object 
 2   label            5418 non-null   int64  
 3   opinion_towards  3519 non-null   object 
 4   sentiment        3519 non-null   object 
 5   text             5418 non-null   object 
 6   target_in_tweet  3519 non-null   float64
 7   seen?            5418 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 338.8+ KB


In [122]:
train_data.head()

,tweet,topic,label,opinion_towards,sentiment,text,target_in_tweet,seen?
0,@Upworthy @HillaryClinton It should be illegal...,"[""hillary"", ""clinton""]",1,1. The tweet explicitly expresses opinion abo...,neg,"[[""illegal"", ""employer"", ""discriminate"", ""work...",1.0,1
1,@HillaryClinton > since trickle down economics...,"[""hillary"", ""clinton""]",0,1. The tweet explicitly expresses opinion abo...,neg,"[[""since"", ""trickle"", ""economics"", ""failed"", ""...",1.0,1
2,Can't wait to get my #YaaasHillary shirt! @Hi...,"[""hillary"", ""clinton""]",1,1. The tweet explicitly expresses opinion abo...,pos,"[[""cant"", ""wait"", ""get"", ""yaaas"", ""hillary"", ""...",1.0,1
3,Too many women in too many countries speak the...,"[""hillary"", ""clinton""]",1,1. The tweet explicitly expresses opinion abo...,neg,"[[""many"", ""women"", ""many"", ""countries"", ""speak...",1.0,1
4,@Conservativeind @cannoneerfour Lets hope! Sh...,"[""hillary"", ""clinton""]",0,3. The tweet is not explicitly expressing opi...,neg,"[[""lets"", ""hope"", ""shes"", ""killer"", ""liar"", ""c...",0.0,1


In [58]:
train_data.tail()

,tweet,topic,label,opinion_towards,sentiment,text,target_in_tweet,seen?
5413,NaN,"[""atheism""]",3,NaN,NaN,"[[""atheists"", ""phonies"", ""morals"", ""allow"", ""o...",NaN,0
5414,NaN,"[""atheism""]",3,NaN,NaN,"[[""way"", ""people"", ""see"", ""belief"", ""god"", ""we...",NaN,0
5415,NaN,"[""atheism""]",3,NaN,NaN,"[[""famous"", ""atheist"", ""compares"", ""clock"", ""b...",NaN,0
5416,NaN,"[""atheism""]",3,NaN,NaN,"[[""see"", ""atheist"", ""todays"", ""organised"", ""mo...",NaN,0
5417,NaN,"[""atheism""]",3,NaN,NaN,"[[""atheist"", ""self"", ""serving"", ""arguments"", ""...",NaN,0


In [110]:
import pandas as pd
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords

def preprocess_tweet(tweet):
    emoticons_happy = set([
        ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3'
    ])
    emoticons_sad = set([
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';('
    ])
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    emoticons = emoticons_happy.union(emoticons_sad)
    # remove URLs
    if isinstance(tweet,str):
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', tweet)
        tweet = re.sub(r'http\S+', '', tweet)
    # remove usernames
        tweet = re.sub('@[^\s]+', '', tweet)
    # remove the # in hashtag and split hashtags
    tweet_toks = str(tweet).split(" ")
    final_tweet_toks = []
    for i in range(len(tweet_toks)):
        if tweet_toks[i].startswith("#"):
            hashtag = tweet_toks[i]
            hashtag = hashtag[1:]
            split_hashtag = re.findall('[0-9]+|[A-Z][a-z]+|[A-Z][A-Z]+|[a-z]+', hashtag)
            final_tweet_toks = final_tweet_toks + split_hashtag
        else:
            final_tweet_toks.append(tweet_toks[i])
    tweet = " ".join(final_tweet_toks)
    # remove emojis
    tweet = emoji_pattern.sub(r'', tweet)
    # convert text to lower-case
    tweet = tweet.lower()
    #Remove any other punctuation
    tweet = [char for char in tweet if char not in string.punctuation]
    tweet = ''.join(tweet)
    #Remove non-ascii characters
    tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)
    #Remove stopwords and emoticons from final word list
    tokens = word_tokenize(tweet)
    stop_words = set(stopwords.words('english'))
    filtered_tweet = []
    for w in tokens:
        if w not in stop_words and w not in emoticons:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)




In [119]:
def fileselection(file_name:str):
    pre = []

    df = pd.read_csv(file_name, encoding='latin1', engine='python')
    
    for row in df["tweet"]:
    #     print(row)
        preprocessed_tweet = preprocess_tweet(row)
        pre.append(preprocessed_tweet)

    df["text"] = pre

    df.to_csv(file_name + "_pre.csv")
    
    return

In [120]:

namelist = ["twitter_testA_seenval","twitter_testCC_seenval","twitter_testDT_seenval","twitter_testFM_seenval","twitter_testHC_seenval","twitter_testLA_seenval"]
mainpath = "C:/Users/aishw/OneDrive/Desktop/Research/Adversarial_Learning/adversarial-learning-for-stance/src/data/twitter_data_naacl/"

for i in namelist:
    fileselection(mainpath+i+"/train.csv")
    fileselection(mainpath+i+"/test.csv")
